In [1]:
import sqlite3
from sqlite3 import Error
import csv
import operator
import pandas as pd
import numpy as np
import time
import datetime
import re
import os

def read_bank_file(fileName):
    if fileName[len(fileName)-3: len(fileName)] == 'csv':
        df = pd.read_csv(fileName)
        print('reading csv')
    else:
        df = pd.read_excel(fileName)
        print('reading xlsx')
    df.drop(df.head(1).index, inplace=True)
    df.drop(['Respondent ID', 'Collector ID', 'Start Date', \
             'IP Address', 'Email Address', 'First Name', 'Last Name', \
             'Custom Data 1'], axis = 1, inplace = True)
    df.columns = ['Inscription_date', 'FirstName', 'LastName', 'Gender', 'BirthDate', 'English_Proficiency', 'French_Proficiency', \
     'Like_Action_Adventure', 'Like_Cards', 'Like_Fighting', 'Like_Indie_Retro', \
     'Like_MOBA', 'Like_MMO', 'Like_Platforms_Puzzle', 'Like_RPG', 'Like_Hach_Splash',
     'Like_Racing', 'Like_Strategy_Management', 'Like_Music_Rythm', 'Like_Stealth', 'Like_Shooters',
     'Like_Sports', 'Like_Survival_Horror', 'Like_VR_AR', 'Like_other', 'Platform_3DS_Switch', 'Platform_Playstation',
     'Platform_XBox', 'Platform_PC', 'Platform_Tablet', 'Platform_Phone', 'Platform_Other', 'Playtime_Console',
     'Playtime_PC', 'Playtime_Tablet', 'Playtime_Phone', 'Videogame_Interests_Y_N', 'Videogame_Interests_Details', 
     'SEM_Playtest_Y_N', 'SEM_Playtest_Details', 'Remove1', 'Remove2', 'Remove3', 'Remove4', 'Remove5', 'Remove6', 
     'Remove7', 'Remove8', 'Email', 'Phone', 'Source_SEM_Website', 'Source_SEM_Staff', 'Source_Kijiji', 'Source_Craigslist',
     'Source_Social_Medias', 'Source_LesPAC', 'Source_Other']
    df.drop(['Remove1', 'Remove2', 'Remove3', 'Remove4', 'Remove5', 'Remove6', 
     'Remove7', 'Remove8'], axis = 1, inplace = True)
    df = df.drop_duplicates(subset=['Email'], keep='last')
    df['Inscription_date'] = df['Inscription_date'].astype('str')
    df['Platform_Other'] = df['Platform_Other'].astype('str')
    return df

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print('From create_connection :', e)
    return None

def create_table_inscriptions(conn, table):
    c = conn.cursor()
    c.execute('CREATE TABLE IF NOT EXISTS {}( \
       Inscription_date TEXT, FirstName TEXT, LastName TEXT, Gender TEXT, BirthDate INT, English_Proficiency INT,\
       French_Proficiency INT, Like_Action_Adventure TEXT, Like_Cards TEXT,\
       Like_Fighting TEXT, Like_Indie_Retro TEXT, Like_MOBA TEXT, Like_MMO TEXT, \
       Like_Platforms_Puzzle TEXT, Like_RPG TEXT, Like_Hach_Splash TEXT, Like_Racing TEXT, \
       Like_Strategy_Management TEXT, Like_Music_Rythm TEXT, Like_Stealth TEXT, \
       Like_Shooters TEXT, Like_Sports TEXT, Like_Survival_Horror TEXT, Like_VR_AR TEXT,\
       Like_other TEXT, Platform_3DS_Switch TEXT, Platform_Playstation TEXT,\
       Platform_XBox TEXT, Platform_PC TEXT, Platform_Tablet TEXT, Platform_Phone TEXT,\
       Platform_Other TEXT, Playtime_Console TEXT, Playtime_PC TEXT, Playtime_Tablet TEXT,\
       Playtime_Phone TEXT, Videogame_Interests_Y_N TEXT,\
       Videogame_Interests_Details TEXT, SEM_Playtest_Y_N TEXT,\
       SEM_Playtest_Details TEXT, Email TEXT, Phone TEXT, Source_SEM_Website TEXT,\
       Source_SEM_Staff TEXT, Source_Kijiji TEXT, Source_Craigslist TEXT,\
       Source_Social_Medias TEXT, Source_LesPAC TEXT, Source_Other TEXT)'.format(table))
    c.execute('CREATE UNIQUE INDEX index_all ON {} (Email)'.format(table))

def write_to_db(df, conn, table):
    repeats = 0
    new_entries = 0
    c = conn.cursor()
    for i in range(df.shape[0]): 
        if isinstance(df.iloc[i,40], str):
            try:
                c.execute("INSERT INTO {}( \
                       'Inscription_date', FirstName, LastName, Gender, BirthDate, English_Proficiency,\
                       French_Proficiency, Like_Action_Adventure, Like_Cards,\
                       Like_Fighting, Like_Indie_Retro, Like_MOBA, Like_MMO,\
                       Like_Platforms_Puzzle, Like_RPG, Like_Hach_Splash, Like_Racing,\
                       Like_Strategy_Management, Like_Music_Rythm, Like_Stealth,\
                       Like_Shooters, Like_Sports, Like_Survival_Horror, Like_VR_AR,\
                       Like_other, Platform_3DS_Switch, Platform_Playstation,\
                       Platform_XBox, Platform_PC, Platform_Tablet, Platform_Phone,\
                       Platform_Other, Playtime_Console, Playtime_PC, Playtime_Tablet,\
                       Playtime_Phone, Videogame_Interests_Y_N,\
                       Videogame_Interests_Details, SEM_Playtest_Y_N,\
                       SEM_Playtest_Details, Email, Phone,  'Source_SEM_Website', 'Source_SEM_Staff', \
                       'Source_Kijiji', 'Source_Craigslist', 'Source_Social_Medias',\
                       'Source_LesPAC', 'Source_Other') \
                          VALUES(?, ?, ?, ?, ?, ?, ?, ?,\
                                 ?, ?, ?, ?, ?, ?, ?, ?,\
                                 ?, ?, ?, ?, ?, ?, ?, ?,\
                                 ?, ?, ?, ?, ?, ?, ?, ?,\
                                 ?, ?, ?, ?, ?, ?, ?, ?,\
                                 ?, ?, ?, ?, ?, ?, ?, ?, ?)".format(table),\
                                 (df.iloc[i,0], df.iloc[i,1].title(), \
                                  df.iloc[i,2].title(), gender(df.iloc[i,3]), df.iloc[i,4],\
                                  df.iloc[i,5], df.iloc[i,6], x(df.iloc[i,7]),\
                                  x(df.iloc[i,8]), x(df.iloc[i,9]), x(df.iloc[i,10]), \
                                  x(df.iloc[i,11]), x(df.iloc[i,12]),x( df.iloc[i,13]), \
                                  x(df.iloc[i,14]),x(df.iloc[i,15]), x(df.iloc[i,16]),\
                                  x(df.iloc[i,17]), x(df.iloc[i,18]), x(df.iloc[i,19]),\
                                  x(df.iloc[i,20]), x(df.iloc[i,21]), x(df.iloc[i,22]), \
                                  x(df.iloc[i,23]), df.iloc[i,24],  x(df.iloc[i,25]), \
                                  x(df.iloc[i,26]), x(df.iloc[i,27]), x(df.iloc[i,28]),\
                                  x(df.iloc[i,29]), x(df.iloc[i,30]), x(df.iloc[i,31]), \
                                  df.iloc[i,32], df.iloc[i,33], \
                                  df.iloc[i,34], playtime_ph(df.iloc[i,35]),\
                                  yn( df.iloc[i,36]), df.iloc[i,37], yn(df.iloc[i,38]), \
                                  df.iloc[i,39], df.iloc[i,40],\
                                  df.iloc[i,41], x(df.iloc[i,42]),\
                                  x(df.iloc[i,43]), x(df.iloc[i,44]), x(df.iloc[i,45]), \
                                  x(df.iloc[i,46]), x(df.iloc[i,47]), df.iloc[i,48]))
                conn.commit() 
            except Exception as e: print('From write_to_db :', e)  #repeats += 1
            new_entries += 1
    print('New Entries = {}, Repeats = {}'.format(new_entries-repeats, repeats))    

def create_table_study(conn, table):
    c = conn.cursor()
    try:
        c.execute('CREATE TABLE IF NOT EXISTS {}( \
           FirstName TEXT, LastName TEXT, Email TEXT,\
           Game_title TEXT, Date TEXT, Notes TEXT, Reinvite TEXT, \
           FOREIGN KEY(Email) REFERENCES All_Participants(Email), \
           UNIQUE (FirstName, LastName, Email, Game_title, Date, Notes, Reinvite)\
           ON CONFLICT REPLACE )'.format(table))
    except Exception as e: print('From create_table_study :', e)    
    c.execute('CREATE UNIQUE INDEX index_email_study ON {} (Email)'.format(table))
            
            
def populate_bank(db_file, fileName):
    df = read_bank_file(fileName)
    conn = create_connection(db_file)
    with conn:
        try:
            create_table_inscriptions(conn, 'All_Participants')
        except Exception as e: print('From populate_bank create_table_inscriptions:', e)
        try:
            write_to_db(df, conn,'All_Participants')
        except Exception as e: print('From populate_bank - write_to_db :', e)    
    conn.close()
     
def populate_study(db_file, fileName, table):
    conn = create_connection(db_file)  
    df = pd.read_csv(fileName, encoding = 'latin1')
    c = conn.cursor()
    for i in range(df.shape[0]):
        lst = []
        for j in range(df.shape[1]):
            if j == 5:
                lst.append(df.iloc[i,j].astype(str)) 
            else:
                lst.append(df.iloc[i,j]) 
        if len(lst[0]) > 0 :
            try:
                c.execute(query(df).format(table),lst)
            except Exception as e: print('From populate_study :', e)
    conn.commit()

def create_study(database, table_name):
    conn = create_connection(database)
    with conn:
        try:
            create_table_study(conn, table_name)
        except Exception as e: print('From create_study :', e)    
    conn.close()    
    
#################### HELPER FUNCTIONS ######################

def query(df):
    att = attributes(df) 
    v = values(df)
    return "%s %s %s %s %s" % ('INSERT INTO {} (', att, ') VALUES (', v, ')')

def attributes(df):
    cols = df.columns
    str = cols[0]
    for i in range(cols.shape[0]):
        if i > 0:
            str = "%s, %s" % (str, cols[i])  
    return str


def values(df):
    temp = ''
    for i  in range(df.columns.shape[0]):
        temp = "%s, %s" % ('?', temp)
    return temp[:-2]    

###############  correct responses ##################
def x(v):
    if type(v) == float:
        return v
    elif len(v) > 0:
        return  'x'
    
def yn(v):
    if len(v)>50:
        return 'No, but playtested'
    elif v[0] == 'N':
        return 'No'
    else:
        return 'Yes'
    
def gender(v):
    return v[0]

def playtime_ph(v):
    if len(str(v)) > 2:
        return int(str(v)[0])
    return v
 
os.chdir('S:/Projects/User Research/DB participants')
print(os.getcwd())    

S:\Projects\User Research\DB participants


In [2]:
# run this line to update the DB with new response
## Make sur the csv file is in here: S:\Projects\User Research\DB participants\Inscriptions_csv


populate_bank('GUR.db','Inscriptions_xlsx/2018-10-04.xlsx')  

reading xlsx
From populate_bank create_table_inscriptions: index index_all already exists
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE 

From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All

From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All

From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All

From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All

From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All

From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All

From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All

From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All_Participants.Email
From write_to_db : UNIQUE constraint failed: All

In [121]:
# ### run once to create the table studies ###
create_study('GUR.db', 'Studies')

In [122]:
# run this line to create a new table, which is populated from the csv form availabe in here
## 'Participants Notes/Participants_notes.csv'
populate_study('GUR.db', 'Participants Notes/Participants_notes.csv', 'Studies')